# Road Traffic Vehicle Counting Exam

In this exam, you will use the YOLO (You Only Look Once) object detection model to analyze a video of road traffic. Your primary task is to count the number of vehicles exiting the road at each of the four exits (located at the top, bottom, left, and right of the frame). The video for this task is provided at `Datasets/YOLO_Exam_Video.mp4`.

## Objectives
- Load and process the video using OpenCV.
- Utilize the YOLO model to detect vehicles in each frame.
- Manipulate the video frames using OpenCV.
- Track vehicles as they move across the video frames.
- Count the number of vehicles exiting via the top street.
- Count the number of vehicles exiting via the bottom street.
- Count the number of vehicles exiting via the left street.
- Count the number of vehicles exiting via the right street.


## Setup Environment

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Before you begin, ensure you have the necessary libraries installed. You will need `opencv`, and `ultralytics` among others.
If these are not installed, you should install them.

In [2]:
!pip install ultralytics
!pip install opencv-python

#Import libres that I need

In [3]:
import cv2
from ultralytics import YOLO

## Load the YOLO Model

You will first need to load the YOLO model, there is a custom model for this exam called `YOLO_Model.pt` under Datasets file ⚠️ PLEASE USE THIS MODEL AND DO NOT USE ANY OTHER MODEL ⚠️.
Write the code to load the YOLO model below:

In [4]:
model = YOLO('/content/drive/MyDrive/py/6- Introduction to computer vision/5- Weekly Project (Sunday)/Dataset/YOLO_Model.pt')

## Prepare the Video Capture

Create a variable to capture the video frames, you can use `cv2.VideoCapture()` to achive this.

In [5]:
cap = cv2.VideoCapture('/content/drive/MyDrive/py/6- Introduction to computer vision/5- Weekly Project (Sunday)/Dataset/YOLO_Video.mp4')

## Get Video Information

You can use `cv2` library to get these information fro the `VideoCapture()` variable you created to extract these information:
* `height`: Video's height.
* `width`: Video's width.
* `fps`: Video's frames.

In [6]:
#Get the vid information
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))

## Prepare Video Writer to Store the Output

Create a variable that uses `cv2.VideoCapture()` to save the video with the bounding boxes, specified Region of Interest rectangle (ROI) and the counted vehicles on each sides. You will need to make the video with the same `fps`, `width`, `height`, and specify the codec and output path of the video.

In [7]:
# The outout video writer
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter('output_videos1.mp4', fourcc, fps, (frame_width, frame_height))



*   Vehicle counters




In [8]:
counters = {'top': 0, 'bottom': 0, 'left': 0, 'right': 0}

## Expermint to get the `(x1, y1, x2, y2)` of the ROI

### Hint
When defining the Regions of Interest (ROIs) for detecting vehicles exiting the road, consider the coordinates that outline the relevant areas. These coordinates can be specified as `(x1, y1, x2, y2)`, where `(x1, y1)` is the top-left corner and `(x2, y2)` is the bottom-right corner of the rectangle.

Experiment with different coordinates to accurately cover the streets where vehicles exit. Visualize these ROIs by drawing rectangles on a frame of the video and adjust the coordinates as needed until you capture the desired areas effectively.

In [9]:
def cheack_exit(x, y, w, h, frame_width, frame_height):
  if y <= 10:
      return 'top'
  elif y + h >= frame_height - 10:
    return 'bottom'
  elif x <= 10:
    return 'left'
  elif x + w >= frame_width - 10:
    return 'right'
  return None

## Process Video Frames and Identify Vehicles Left the Road Form Each Street

For each frame in the video, use the YOLO model to detect and track vehicles. You will need to write a loop that processes each frame and applies the YOLO model.

In each frame, after detecting the vehicles, determine whether they exited the road via the top, bottom, left, or right street. You can use the positions of the bounding boxes provided by YOLO to do this.

- The video should display bounding boxes around the detected objects.
- The video should display your name on the top.
- It should display the calculated center of each vehicle.
- The video should display the confidence score, alongside the object ID and class ID of each detected and tracked object.
- The video should display the rectangles representing the regions where you count the vehicles that have crossed and exited.
- The video should display the number of vehicles that exited via the top street.
- The video should display the number of vehicles that exited via the bottom street.
- The video should display the number of vehicles that exited via the left street.
- The video should display the number of vehicles that exited via the right street.

In [11]:
while cap.isOpened():
  ret, frame = cap.read()
  if not ret:
    break

  #Detect vehical in the video
  results = model(frame)[0]
  boxes = results.boxes.data.cpu().numpy()

  for i, box in enumerate(boxes):
    x1, y1, x2, y2, score, class_id = map(int, box[:6])
    if score >= 0.5 and class_id in [2, 3, 5, 7]:
      exit_dir = cheack_exit(x1, y1, x2-x1, y2-y1, frame.shape[1], frame.shape[0])


      if exit_dir:
        counters[exit_dir] += 1
        cv2.putText(frame, f'Exit: {exit_dir}', (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)


        #draw bounding box
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)

        #Display conf and ID class
        label = f'ID:{i} conf:{score:.2f}'
        cv2.putText(frame, label, (x1, y1 - 30), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)


     # Display the vehicle counts
    for j, (direction, count) in enumerate(counters.items()):
        cv2.putText(frame, f'{direction.capitalize()}: {count}', (10, 30 + j * 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

    out.write(frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
      break

cap.release()
out.release()
cv2.destroyAllWindows()



0: 384x640 3 buss, 97 cars, 1 truck, 5 vans, 62.9ms
Speed: 3.4ms preprocess, 62.9ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 buss, 97 cars, 1 truck, 4 vans, 62.8ms
Speed: 2.2ms preprocess, 62.8ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 buss, 95 cars, 1 truck, 4 vans, 62.9ms
Speed: 3.4ms preprocess, 62.9ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 buss, 96 cars, 1 truck, 3 vans, 62.8ms
Speed: 2.2ms preprocess, 62.8ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 buss, 96 cars, 1 truck, 3 vans, 62.9ms
Speed: 3.9ms preprocess, 62.9ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 buss, 98 cars, 1 truck, 4 vans, 62.8ms
Speed: 4.2ms preprocess, 62.8ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 buss, 97 cars, 1 truck, 4 vans, 62.9ms
Speed: 3.3ms preprocess, 62.9ms inference, 1.4m

## Save and Submit Your Work

Submit both the notebook and the output video